In [1]:
import numpy as np
import sympy as sym
import math as mt

In [2]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [3]:
def GetLaguerreRecursive(n,x):

    if n==0:
        poly = sym.Pow(1,1)
    elif n==1:
        poly = 1- x
    else:
        poly = ((2*n-1-x)*GetLaguerreRecursive(n-1,x)-(n-1)*GetLaguerreRecursive(n-2,x))/n
   
    return sym.simplify(poly)

In [4]:
GetLaguerreRecursive(0,x)

1

In [5]:
GetLaguerreRecursive(1,x)

1 - x

In [6]:
GetLaguerreRecursive(2,x) #equivalente a (x^2-4x+2)/2

(x - 3)*(x - 1)/2 - 1/2

In [7]:
GetLaguerreRecursive(3,x) #equivalente a (-x^3+9x^2-18x+6)/6. Por lo que ya comprobamos que esta bien el anterior metodo

-x**3/6 + 3*x**2/2 - 3*x + 1

In [8]:
def GetDLaguerre(n,x):
    Pn = GetLaguerreRecursive(n,x)
    return sym.diff(Pn,x,1)

In [9]:
GetDLaguerre(3,x)#coincide con lo calculado

-x**2/2 + 3*x - 3

In [10]:
def GetNewton(f,df,xn,itmax=10000,precision= 1e-6):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [11]:
def GetRootsGLag(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)
    
        croot = np.round( root, tolerancia )
        
        if croot not in Roots:
            Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [12]:
def GetAllRootsGLag(n):

    xn = np.linspace(0, n+(n-1)*np.sqrt(n),100)
    
    Laguerre = []
    DLaguerre = []
    
    for i in range(n+1):
        Laguerre.append(GetLaguerreRecursive(i,x))
        DLaguerre.append(GetDLaguerre(i,x))
    
    poly = sym.lambdify([x],Laguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLaguerre[n],'numpy')
    Roots = GetRootsGLag(poly,Dpoly,xn)
    
    return Roots

In [13]:
GetAllRootsGLag(5)

array([ 0.26356032,  1.41340306,  3.59642577,  7.08581001, 12.64080084])

In [14]:
def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)
    
    Laguerre = []
    
    for i in range(n+2):
        Laguerre.append(GetLaguerreRecursive(i,x))
        
    poly = sym.lambdify([x],Laguerre[n+1],'numpy')
    #Dpoly = sym.lambdify([x],DLaguerre[n],'numpy')
    Weights = Roots/(((n+1)**2)*(poly(Roots)**2))
    
    return Weights

In [15]:
raices = GetAllRootsGLag(5)
raices

array([ 0.26356032,  1.41340306,  3.59642577,  7.08581001, 12.64080084])

In [16]:
pesos = GetWeightsGLag(5)
pesos

array([5.21755611e-01, 3.98666811e-01, 7.59424497e-02, 3.61175868e-03,
       2.33699724e-05])

In [17]:
funcion = lambda x: x**2


In [18]:
I = 0
for i in range(5):

    I += pesos[i]*funcion(raices[i])

In [19]:
I

2.000000000146056